**2. CARGA Y TRATAMIENTOS DE DATOS**

In [1]:
##############IMPORTANDO LIBRERIAS################
import pandas as pd
import re

##########IMPORTANDO ARCHIVOS EXCEL CONTACTOS Y LLAVES##########
contactos = pd.ExcelFile(r'contactos1.xlsx').parse(0)
llaves = pd.ExcelFile(r'llaves.xlsx').parse(0)
print('Importado', contactos.shape[0],'numeros de celular')

#################DUPLICADOS Y ESPACIOS VACIOS ENTRE NUEMROS################
x=contactos.shape[0]
contactos = contactos.drop_duplicates()
y=contactos.shape[0]
z=x-y
print(z,'duplicados encontrados y removidos')

##REMOVER
def espacios(texto):
    texto = re.sub(r'[\s]+','',texto)
    return texto
contactos['cel']=contactos['cel'].astype(str).apply(espacios)

#OBJETOS LIST
mensaje = list(llaves.mensaje)
mensaje = [x for x in mensaje if str(x) != 'nan']
ruta_imagen = list(llaves.ruta_imagen)
ruta_imagen = [x for x in ruta_imagen if str(x) != 'nan']
ruta_video = list(llaves.ruta_video)
ruta_video = [x for x in ruta_video if str(x) != 'nan']
ruta_documento = list(llaves.ruta_documento)
ruta_documento = [x for x in ruta_documento if str(x) != 'nan']

contactos['cel'] = contactos['codigo_pais'].astype(str) + contactos['cel']
contactos['cel']=contactos['cel'].str.replace('.0','',regex=False)

Importado 1360 numeros de celular
4 duplicados encontrados y removidos


**3. ENVIO DE MENSAJES**

**3.1 Importando librerías desde Selenium**

In [2]:
#Para llamar al chromedriver
from selenium import webdriver

#Espera un tiempo hasta que cargue el elemento ui en el DOM
from selenium.webdriver.support.ui import WebDriverWait

#Activa las capacidades para interactuar con el chromedriver
from selenium.webdriver.chrome.options import Options

#Enviar informacion a la ui
from selenium.webdriver.common.keys import Keys
import time
import pathlib
import pandas as pd

df = pd.DataFrame(columns=['cel','estado'])

print('Abriendo WhatsApp Web')

chrome_options=Options()
scriptDirectory = pathlib.Path().absolute()
chrome_options.add_argument(f"--user-data-dir={scriptDirectory}\\userdata")
driver=webdriver.Chrome('chromedriver.exe',options=chrome_options)
driver.get('https://web.whatsapp.com')

Abriendo WhatsApp Web


**3.2 Inspeccionando WhatsApp Web**

//*[@id="main"]/footer/div[1]/div[2]/div/div[2]

**3.3 Armando el código para enviar mensajes**

**4. ENVIO DE IMAGENES Y VIDEOS**

**4.1 Inspeccionando WhatsApp Web**

$\color{#797979}{\text{xpath boton adjuntar:}}$
//div[@title = "Adjuntar"]

$\color{#797979}{\text{xpath imagen box:}}$
//input[@accept="image/*,video/mp4,video/3gpp,video/quicktime"]

$\color{#797979}{\text{xpath boton envio:}}$
//*[@id="app"]/div/div/div[2]/div[2]/span/div/span/div/div/div[2]/span/div/div

**4.2 Armando el código para enviar imágenes y videos**

In [3]:
a=0
cel = contactos['cel']
for i in cel:
    print('-----------------------------------------------')
    print('Enviando mensaje a: +',i)
    link = 'https://web.whatsapp.com/send?phone='+i
    driver.get(link)
    a=a+1
    df.at[a, 'cel'] = i
    try:
        for j in mensaje:
            print('Escribiendo mensaje:',j)
            input_xpath = '//*[@id="main"]/footer/div[1]/div[2]/div/div[2]'
            input_box = WebDriverWait(driver,40).until(lambda driver: driver.find_element_by_xpath(input_xpath))
            time.sleep(2)
            input_box.send_keys(j + Keys.ENTER)
            time.sleep(2)
            print('Mensaje enviado')
            df.at[a, 'estado'] ='Mensaje enviado satisfactoriamente'
        if ruta_imagen:
            for k in ruta_imagen:
                print('Enviando imagen')
                attach_button_xpath = '//div[@title = "Adjuntar"]'
                attach_button = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(attach_button_xpath))
                time.sleep(5)
                attach_button.click()
                image_box_xpath = '//input[@accept="image/*,video/mp4,video/3gpp,video/quicktime"]'
                image_box = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(image_box_xpath))
                image_box.send_keys(k)
                time.sleep(5)
                send_button = WebDriverWait(driver,40).until(lambda driver: driver.find_element_by_xpath('//*[@id="app"]/div[1]/div[1]/div[2]/div[2]/span/div[1]/span/div[1]/div/div[2]/div/div[2]/div[2]/div/div'))
                send_button.click()
                time.sleep(5)
                print('Imagen enviada')
                df.at[a, 'estado'] ='Mensaje enviado satisfactoriamente'
        if ruta_video:
            for k in ruta_video:
                print('Enviando video')
                attach_button_xpath = '//div[@title = "Adjuntar"]'
                attach_button = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(attach_button_xpath))
                time.sleep(5)
                attach_button.click()
                image_box_xpath = '//input[@accept="image/*,video/mp4,video/3gpp,video/quicktime"]'
                image_box = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(image_box_xpath))
                image_box.send_keys(k)
                time.sleep(2)
                send_button = WebDriverWait(driver,40).until(lambda driver: driver.find_element_by_xpath('//*[@id="app"]/div/div/div[2]/div[2]/span/div/span/div/div/div[2]/span/div/div'))
                send_button.click()
                time.sleep(2)
                print('Video enviado')
                df.at[a, 'estado'] ='Mensaje enviado satisfactoriamente'
        if ruta_documento:
            for k in ruta_documento:
                print('Enviando documento')
                attach_button_xpath = '//div[@title = "Adjuntar"]'
                attach_button = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(attach_button_xpath))
                time.sleep(2)
                attach_button.click()
                image_box_xpath = '//input[@accept="*"]'
                image_box = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(image_box_xpath))
                image_box.send_keys(k)
                time.sleep(3)
                send_button = WebDriverWait(driver,40).until(lambda driver: driver.find_element_by_xpath('//*[@id="app"]/div[1]/div[1]/div[2]/div[2]/span/div[1]/span/div[1]/div/div[2]/div/div[2]/div[2]/div/div'))
                send_button.click()
                time.sleep(20)
                print('Documento enviado')                
                df.at[a, 'estado'] ='Mensaje enviado satisfactoriamente'               
    except:
        print('Este numero no tiene WhatsApp: +',i)
        df.at[a, 'estado'] ='Numero sin WhatsApp'
        print('-----------------------------------------------')
            



-----------------------------------------------
Enviando mensaje a: + 56966836510
Enviando imagen
Este numero no tiene WhatsApp: + 56966836510
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56966879691
Enviando imagen
Este numero no tiene WhatsApp: + 56966879691
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56967083435
Enviando imagen
Este numero no tiene WhatsApp: + 56967083435
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56967131334
Enviando imagen
Este numero no tiene WhatsApp: + 56967131334
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56967195576
Enviando imagen
Este numero no tiene WhatsApp: + 56967195576
-----------------------------------------------
---------------------------------------------

Enviando imagen
Este numero no tiene WhatsApp: + 56971077448
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56971351526
Enviando imagen
Este numero no tiene WhatsApp: + 56971351526
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56971356959
Enviando imagen
Este numero no tiene WhatsApp: + 56971356959
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56971359979
Enviando imagen
Este numero no tiene WhatsApp: + 56971359979
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56971375135
Enviando imagen
Este numero no tiene WhatsApp: + 56971375135
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56971381269
Enviando imagen
Este numero no tiene WhatsApp

Enviando imagen
Este numero no tiene WhatsApp: + 56974956011
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56974956055
Enviando imagen
Este numero no tiene WhatsApp: + 56974956055
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56974964190
Enviando imagen
Este numero no tiene WhatsApp: + 56974964190
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56974968171
Enviando imagen
Este numero no tiene WhatsApp: + 56974968171
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56974972518
Enviando imagen
Este numero no tiene WhatsApp: + 56974972518
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56974980147
Enviando imagen
Este numero no tiene WhatsApp

Enviando imagen
Este numero no tiene WhatsApp: + 56976599464
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56976632661
Enviando imagen
Este numero no tiene WhatsApp: + 56976632661
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56976661223
Enviando imagen
Este numero no tiene WhatsApp: + 56976661223
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56976679897
Enviando imagen
Este numero no tiene WhatsApp: + 56976679897
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56976680242
Enviando imagen
Este numero no tiene WhatsApp: + 56976680242
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56976693479
Enviando imagen
Este numero no tiene WhatsApp

Enviando imagen
Este numero no tiene WhatsApp: + 56977884228
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56977911312
Enviando imagen
Este numero no tiene WhatsApp: + 56977911312
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56977926730
Enviando imagen
Este numero no tiene WhatsApp: + 56977926730
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56977941260
Enviando imagen
Este numero no tiene WhatsApp: + 56977941260
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56977949547
Enviando imagen
Este numero no tiene WhatsApp: + 56977949547
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56977973539
Enviando imagen
Este numero no tiene WhatsApp

Enviando imagen
Este numero no tiene WhatsApp: + 56979667788
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56979670250
Enviando imagen
Este numero no tiene WhatsApp: + 56979670250
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56979771207
Enviando imagen
Este numero no tiene WhatsApp: + 56979771207
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56979796466
Enviando imagen
Este numero no tiene WhatsApp: + 56979796466
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56979866033
Enviando imagen
Este numero no tiene WhatsApp: + 56979866033
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56979866326
Enviando imagen
Este numero no tiene WhatsApp

Enviando imagen
Este numero no tiene WhatsApp: + 56982282724
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982283664
Enviando imagen
Este numero no tiene WhatsApp: + 56982283664
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982289818
Enviando imagen
Este numero no tiene WhatsApp: + 56982289818
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982289878
Enviando imagen
Este numero no tiene WhatsApp: + 56982289878
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982290460
Enviando imagen
Este numero no tiene WhatsApp: + 56982290460
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982294143
Enviando imagen
Este numero no tiene WhatsApp

Enviando imagen
Este numero no tiene WhatsApp: + 56982935827
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982936192
Enviando imagen
Este numero no tiene WhatsApp: + 56982936192
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982944458
Enviando imagen
Este numero no tiene WhatsApp: + 56982944458
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982945124
Enviando imagen
Este numero no tiene WhatsApp: + 56982945124
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982945966
Enviando imagen
Este numero no tiene WhatsApp: + 56982945966
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56982990430
Enviando imagen
Este numero no tiene WhatsApp

Enviando imagen
Este numero no tiene WhatsApp: + 56998447227
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56984476006
Enviando imagen
Este numero no tiene WhatsApp: + 56984476006
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56984482204
Enviando imagen
Este numero no tiene WhatsApp: + 56984482204
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56984495452
Enviando imagen
Este numero no tiene WhatsApp: + 56984495452
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56984497265
Enviando imagen
Este numero no tiene WhatsApp: + 56984497265
-----------------------------------------------
-----------------------------------------------
Enviando mensaje a: + 56984499888
Enviando imagen
Este numero no tiene WhatsApp

WebDriverException: Message: chrome not reachable
  (Session info: chrome=92.0.4515.159)


**5. ENVIO DE DOCUMENTOS**

**5.1 Inspeccionando WhatsApp Web**

$\color{#797979}{\text{xpath boton adjuntar:}}$
//div[@title = "Adjuntar"]

$\color{#797979}{\text{xpath documentos box:}}$
//input[@accept="*"]

$\color{#797979}{\text{xpath boton envio:}}$
//*[@id="app"]/div/div/div[2]/div[2]/span/div/span/div/div/div[2]/span/div/div

**5.2 Armando el código para enviar documentos**

In [ ]:
a=0
cel = contactos['cel']
for i in cel:
    print('-----------------------------------------------')
    print('Enviando mensaje a: +',i)
    link = 'https://web.whatsapp.com/send?phone='+i
    driver.get(link)
    a=a+1
    df.at[a, 'cel'] = i
    try:
        for j in mensaje:
            print('Escribiendo mensaje:',j)
            input_xpath = '//*[@id="main"]/footer/div[1]/div[2]/div/div[2]'
            input_box = WebDriverWait(driver,40).until(lambda driver: driver.find_element_by_xpath(input_xpath))
            time.sleep(2)
            input_box.send_keys(j + Keys.ENTER)
            time.sleep(2)
            print('Mensaje enviado')
            df.at[a, 'estado'] ='Mensaje enviado satisfactoriamente'
        if ruta_imagen:
            for k in ruta_imagen:
                print('Enviando imagen')
                attach_button_xpath = '//div[@title = "Adjuntar"]'
                attach_button = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(attach_button_xpath))
                time.sleep(2)
                attach_button.click()
                image_box_xpath = '//input[@accept="image/*,video/mp4,video/3gpp,video/quicktime"]'
                image_box = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(image_box_xpath))
                image_box.send_keys(k)
                time.sleep(3)
                send_button = WebDriverWait(driver,40).until(lambda driver: driver.find_element_by_xpath('//*[@id="app"]/div[1]/div[1]/div[2]/div[2]/span/div[1]/span/div[1]/div/div[2]/div/div[2]/div[2]/div/div'))
                send_button.click()
                time.sleep(8)
                print('Imagen enviada')
                df.at[a, 'estado'] ='Mensaje enviado satisfactoriamente'
        if ruta_video:
            for k in ruta_video:
                print('Enviando video')
                attach_button_xpath = '//div[@title = "Adjuntar"]'
                attach_button = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(attach_button_xpath))
                time.sleep(4)
                attach_button.click()
                image_box_xpath = '//input[@accept="image/*,video/mp4,video/3gpp,video/quicktime"]'
                image_box = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(image_box_xpath))
                image_box.send_keys(k)
                time.sleep(4)
                send_button = WebDriverWait(driver,40).until(lambda driver: driver.find_element_by_xpath('//*[@id="app"]/div/div/div[2]/div[2]/span/div/span/div/div/div[2]/span/div/div'))
                send_button.click()
                time.sleep(20)
                print('Video enviado')
                df.at[a, 'estado'] ='Mensaje enviado satisfactoriamente'
        if ruta_documento:
            for k in ruta_documento:
                print('Enviando documento')
                attach_button_xpath = '//div[@title = "Adjuntar"]'
                attach_button = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(attach_button_xpath))
                time.sleep(4)
                attach_button.click()
                doc_box_xpath = '//input[@accept="*"]'
                doc_box = WebDriverWait(driver,20).until(lambda driver: driver.find_element_by_xpath(doc_box_xpath))
                doc_box.send_keys(k)
                time.sleep(4)
                send_button = WebDriverWait(driver,40).until(lambda driver: driver.find_element_by_xpath('//*[@id="app"]/div/div/div[2]/div[2]/span/div/span/div/div/div[2]/span/div/div'))
                send_button.click()
                time.sleep(20)
                print('Documento enviado')
                df.at[a, 'estado'] ='Mensaje enviado satisfactoriamente'             
                
                
    except:
        print('Este numero no tiene WhatsApp: +',i)
        df.at[a, 'estado'] ='Numero sin WhatsApp'
        print('-----------------------------------------------')

print('_______________________')
print('Mensaje de WS enviados. Exportando reporte')
from datetime import datetime
fechahoy=datetime.now()
FFH=fechahoy.strftime("%Y%m%d_%H%M")
nombre=FFH+'_reporte'
df.to_excel('{}.xlsx'.format(nombre),index=False)
print('Reporte exportado')
print('Cerrando programa')
driver.quit()
        
        
        
        
        
        
        
        
        
        

